# runBJetEnergyPeak.py converted to notebook
## By Anthony Vizcaíno Aportela
### Requires python3 and the imports below
### If running python2, replace (or comment out) print statements and replace line 7 of `main()` function.

### any place where there is a (?) means I'm not entirely sure if it's true.

Import statements. Some of these aren't actually used in this notebook(?) but I'm afraid to change these in case it breaks everything.

In [1]:
import optparse
import os, sys
import json as js
import pickle
import ROOT
from subprocess import Popen, PIPE

Welcome to JupyROOT 6.24/06


Define interesting histograms in a dictionary. The function `fillHistos()` depends heavily on this dictionary. If you edit the dictionary, make sure to edit `fillHistos()` accordingly.

In [2]:
histos = { 
    'nvtx'    :ROOT.TH1F('nvtx'    ,';Vertex multiplicity; Events'                  ,30,0,30),
    'nbtags'  :ROOT.TH1F('nbtags'  ,';b-tag multiplicity; Events'                   ,5,0,5),
    'nleptons':ROOT.TH1F('nleptons',';Lepton multiplicity; Events'                  ,5,0,5),
    'bjeten'  :ROOT.TH1F('bjeten'  ,';Energy [GeV]; Jets'                           ,30,0,300),
    'bmjeteta':ROOT.TH1F('bmjeteta',';#eta(b matched jet); Events'                  ,24,-2.4,2.4),
    'lep0pt'  :ROOT.TH1F('lep0pt'  ,';Leading Lepton Transverse Momentum; Events'   ,25,0,250),
    'lep1pt'  :ROOT.TH1F('lep1pt'  ,';Subleading Lepton Transverse Momentum; Events',20,0,200),
    'bjeteta' :ROOT.TH1F('bjeteta' ,';#eta; Jets'                                   ,50,-3,3),
    'bjetenls':ROOT.TH1F('bjetenls',';log(E);  1/E dN_{b jets}/dlog(E)'             ,20,3.,7.),
    'metpt'   :ROOT.TH1F('metpt'   ,';MET [GeV]; Jets'                              ,55,0.,1100.),
    'elpt'    :ROOT.TH1F('elpt'    ,';electron pt [GeV]; electrons'                 ,40,0.,400.),
    'eleta'   :ROOT.TH1F('eleta'   ,';#eta; electrons'                              ,50,-3,3),
    'mupt'    :ROOT.TH1F('mupt'    ,';muon pt [GeV]; muons'                         ,40,0.,400.),
    'mueta'   :ROOT.TH1F('mueta'   ,';#eta; muons'                                  ,50,-3,3)
    }

Set proper weights(?) for histograms and set directory such that histograms are not erased from memory(?). Input is dictionary of histogram objects. Action is to mutate histogram objects in dictionary.

In [3]:
def histoSettings(histos):
    for key in histos:
        histos[key].Sumw2()
        histos[key].SetDirectory(0)

Open root file, get tree named 'data', and count number of entries in tree. Input is root file URL string. Action is to return file object, 'data' tree object, and number of entries.

In [4]:
def treeOpener(inFileURL):
    fIn          = ROOT.TFile.Open(inFileURL)
    tree         = fIn.Get('data')
    totalEntries = tree.GetEntriesFast()
    
    return fIn, tree, totalEntries

Tests if an event has one or two B jets with the following requirements:
* $P_{T}>$ 30
* $\eta<2$ .4
* IVF $>$ .8484
* Contains $b$ quarks(?)

If it does not, then this function signals in the `mainSelectionLoop()` function to skip the rest of the loop.

In [5]:
def twoJetReq(nJets, nBtags, tree, taggedJetsP4, matchedJetsP4):
    for ij in range(0,tree.nJet):
        #get the kinematics and select the jet
        jp4 = ROOT.TLorentzVector()
        jp4.SetPtEtaPhiM(tree.Jet_pt[ij],tree.Jet_eta[ij],tree.Jet_phi[ij],tree.Jet_mass[ij])
        if jp4.Pt() < 30 or ROOT.TMath.Abs(jp4.Eta()) > 2.4 : continue

        #count selected jet
        nJets += 1

        #save P4 for b-tagged jet
        if tree.Jet_CombIVF[ij]>0.8484: # medium cut
            nBtags += 1
            taggedJetsP4.append(jp4)
            if abs(tree.Jet_flavour[ij]) == 5:
                matchedJetsP4.append(jp4)

    if nJets < 2 : raise Exception('not enough jets!')
    if nBtags != 1 and nBtags != 2 : raise Exception('wrong number of Btags!')
    return nJets, nBtags

Tests if an event has two leptons with the following requirements:
* $P_{T}>$ 20
* $\eta<$ 2.4

If it does not, then this function signals in the `mainSelectionLoop()` function to skip the rest of the loop.

In [6]:
def lepReq(nLeptons, tree, leptonsP4):
    
    for ij in range(0,tree.nLepton):
            
        #get the kinematics and select the lepton      
        lp4 = ROOT.TLorentzVector()
        lp4.SetPtEtaPhiM(tree.Lepton_pt[ij],tree.Lepton_eta[ij],tree.Lepton_phi[ij],0)
        if lp4.Pt()<20 or ROOT.TMath.Abs(lp4.Eta())>2.4 : continue

        #count selected leptons
            
        nLeptons +=1
            
        leptonsP4.append(lp4)

    if nLeptons<2 : raise Exception('not enough leptons!')
    return nLeptons

Generator level weight for MC

In [7]:
def wgtMut(evWgt, xsec, tree):
    if xsec              : evWgt  = xsec*tree.LepSelEffWeights[0]*tree.PUWeights[0]
    if tree.nGenWeight>0 : evWgt *= tree.GenWeights[0]
    return evWgt

Fill Histograms. This function depends heavily on the histos dictionary. If you edit the histos dictionary, make sure to edit this function accordingly.

In [8]:
def fillHistos(nJets, nBtags, nLeptons, 
               taggedJetsP4, leptonsP4, matchedJetsP4,
               evWgt, histos, xsec, tree):
    
    #ready to fill the histograms
    histos['nvtx'].Fill(tree.nPV,evWgt)
    histos['nbtags'].Fill(nBtags,evWgt)
    histos['nleptons'].Fill(nLeptons,evWgt)
    histos['metpt'].Fill(tree.MET_pt,evWgt)

    #use up to two leading b-tagged jets
    for ij in range(0,len(taggedJetsP4)):
        if ij>1 : break
        histos['bjeten'].Fill(taggedJetsP4[ij].E(),evWgt)
        histos['bjetenls'].Fill(ROOT.TMath.Log(taggedJetsP4[ij].E()),evWgt/taggedJetsP4[ij].E())
        histos['bjeteta'].Fill(taggedJetsP4[ij].Eta(),evWgt)

    for ij in range(0,len(matchedJetsP4)):
        histos['bmjeteta'].Fill(matchedJetsP4[ij].Eta(),evWgt)

    for ij in range(0,len(leptonsP4)):
        if ij>1 : break
        lid=abs(tree.Lepton_id[ij])
        if lid!=11 and lid!=13:
            raise Exception("Wrong lepton id!")

        if ij == 0: histos['lep0pt'].Fill(leptonsP4[ij].Pt(),evWgt)
        if ij == 1: histos['lep1pt'].Fill(leptonsP4[ij].Pt(),evWgt)

        #hard-coded masses for electrons and muons
        #lmass=0.00051 if lid==11 else 0.106
        ltag='el' if lid==11 else 'mu'
        histos[ltag+'pt'].Fill(leptonsP4[ij].Perp(),evWgt)
        histos[ltag+'eta'].Fill(leptonsP4[ij].Eta(),evWgt)

Main selection loop

In [9]:
def mainSelectionLoop(fIn, tree, totalEntries, 
                      inFileURL, outFileURL, histos, xsec):
    for i in range(0,totalEntries):
        tree.GetEntry(i)
        if i%100==0 : sys.stdout.write('\r [ %d/100 ] done' %(int(float(100.*i)/float(totalEntries))) )
        
        nJets, nBtags, nLeptons = 0, 0, 0
        taggedJetsP4 = []
        leptonsP4 = []
        matchedJetsP4 = []            

        #require at most two Btaged jets
        try: nJets, nBtags = twoJetReq(tree, taggedJetsP4, matchedJetsP4)
        except: continue
               
        #require two leptons with the right properties
        try: nLeptons = lepReq(tree, leptonsP4)
        except: continue

        #generator level weight only for MC
        evWgt = 1.0
        evWgt = wgtMut(evWgt, xsec, tree)

        #fill histograms
        fillHistos(nJets, nBtags, nLeptons, 
               taggedJetsP4, leptonsP4, matchedJetsP4,
               evWgt, histos, xsec, tree) 

    fIn.Close()


Save histograms to file

In [10]:
def histoSaver(outFileURL, histos):
    fOut=ROOT.TFile.Open(outFileURL,'RECREATE')
    for key in histos: histos[key].Write()
    fOut.Close()

Perform the analysis on a single file

In [11]:
def runBJetEnergyPeak(inFileURL, outFileURL, histos, xsec=None):

    print('...analysing ' + inFileURL)

    histoSettings(histos)

    fIn, tree, totalEntries = treeOpener(inFileURL)
    
    mainSelectionLoop(fIn, tree, totalEntries, inFileURL, outFileURL, histos, xsec)

    histoSaver(outFileURL, histos)

For running in parallel

In [12]:
def runBJetEnergyPeakPacked(args):
      
    try:
        return runBJetEnergyPeak(inFileURL  = args[0],
                                 outFileURL = args[1],
                                 histos     = args[2],
                                 xsec       = args[3])
    except :
        #print(50*'<')
        print("  Problem  (%s) with %s continuing without"%(sys.exc_info()[1],args[0]))
        #print(50*'<')
        return False

Main running function

In [13]:
def main(histos, json=None, inDir=None, outDir='analysis', njobs = 0):
    
    #read list of samples
    jsonFile = open(json,'r')
    ## replace samplesList with the following if running python2
    #samplesList = js.load(jsonFile,encoding='utf-8').items()
    samplesList=js.load(jsonFile).items()
    jsonFile.close()
    
    #prepare output
    if len(outDir)==0    : outDir='./'
    os.system('mkdir -p ' + outDir)
    
    #create the analysis jobs
    taskList = []
    for sample, sampleInfo in samplesList: 
        inFileURL  = 'root://cmseos.fnal.gov//%s/%s.root' % (inDir,sample)
        #if not os.path.isfile(inFileURL): continue
        xsec = sampleInfo[0] if sampleInfo[1] == 0 else None        
        outFileURL = '%s/%s.root' % (outDir,sample)
        taskList.append( (inFileURL, outFileURL, histos, xsec) )

        #run the analysis jobs
    if njobs == 0:
        for inFileURL, outFileURL, histos, xsec in taskList:
            runBJetEnergyPeak(inFileURL  = inFileURL,
                              outFileURL = outFileURL, 
                              histos     = histos,
                              xsec       = xsec)
    else:
        from multiprocessing import Pool
        pool = Pool(njobs)
        pool.map(runBJetEnergyPeakPacked,taskList)    

    print('Analysis results are available in ' + outDir)
    

    

Cell for running

In [14]:
inDir  = '/store/user/cmsdas/2022/long_exercises/TopMass' 
json   = 'data/samples_Run2016_25ns.json' 
outDir = 'test' 
njobs  = 4

main(histos, json, inDir, outDir, njobs)

...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/TTJets.root...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/WW.root...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/WJets.root


...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/atW.root
 [ 0/100 ] done...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/tW.root
 [ 62/100 ] done...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/DY.root
 [ 2/100 ] donee...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/MuonEG_2016B.root
 [ 93/100 ] done...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/MuonEG_2016C.root
 [ 3/100 ] donee...analysing root://cmseos.fnal.gov///store/user/cmsdas/2022/long_exercises/TopMass/MuonEG_2016D.root
 [ 76/100 ] done...analysing root://cmseos.fnal.gov///s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 [ 99/100 ] doneAnalysis results are available in test
